In [1]:
from datetime import datetime
from pprint import pprint
import pandas as pd

import sys
sys.path.append('..')
import doctable as dt

In [2]:
schema = (
    # standard id column
    #SQLAlchemy: Column('id', Integer, primary_key = True, autoincrement=True), 
    ('integer', 'id',dict(primary_key=True, autoincrement=True)),

    # make a category column with two options: "FICTION" and "NONFICTION"
    #SQLAlchemy: Column('title', String,)
    
    ('string', 'category', dict(nullable=False)),
    

    # make a non-null title column
    #SQLAlchemy: Column('title', String,)
    ('string', 'title', dict(nullable=False)),

    # make an abstract where the default is an empty string instead of null
    #SQLAlchemy: Column('abstract', String, default='')
    ('string', 'abstract',dict(default='')),

    # make an age column where age must be greater than zero
    #SQLAlchemy: Column('abstract', Integer)
    ('integer', 'age'),
 

    # make a column that keeps track of column updates
    #SQLAlchemy: Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
    ('datetime', 'updated_on',  dict(default=datetime.now, onupdate=datetime.now)),

    # make a string column with max of 500 characters
    #SQLAlchemy: Column('abstract', String, default='')
    ('string', 'text',dict(),dict(length=500)),
    
    
    
    ##### Constraints #####
    
    #SQLAlchemy: CheckConstraint('category in ("FICTION","NONFICTION")', name='salary_check')
    ('check_constraint', 'category in ("FICTION","NONFICTION")', dict(name='salary_check')),
    
    #SQLAlchemy: CheckConstraint('age > 0')
    ('check_constraint', 'age > 0'),
    
    # make sure each category/title entry is unique
    #SQLAlchemy:  UniqueConstraint('category', 'title', name='work_key')
    ('unique_constraint', ['category','title'], dict(name='work_key')),
    
    
    ##### Indexes ######
    
    # make index table
    # SQLAlchemy: Index('ind0', 'category', 'title', unique=True)
    ('index', 'ind0', ('category','title'),dict(unique=True)),
    
    
    # make this id a foreign key to another table column (CAN'T SHOW HERE)
    # SQLAlchemy: ForeignKeyConstraint(['id'], ['invoice.invoice_id',], onupdate="CASCADE", ondelete="CASCADE")
    #((['id'],['othertablerefid']), 'foreignkey_constraint',dict(),dict(onupdate="CASCADE", ondelete="CASCADE")),
    
    # try out custom data types
    # TokensType and ParagraphsType are defined in doctable/coltypes.py
    # SQLAlchemy: Column('tokenized', TokensType), Column('sentencized', ParagraphsType)
    ('pickle','tokenized'),
)
md = dt.DocTable(schema, verbose=True)
#pprint(md.schemainfo)
pd.DataFrame(md.schemainfo)

,name,type,nullable,default,autoincrement,primary_key
0,id,INTEGER,False,None,auto,1
1,category,VARCHAR,False,None,auto,0
2,title,VARCHAR,False,None,auto,0
3,abstract,VARCHAR,True,None,auto,0
4,age,INTEGER,True,None,auto,0
5,updated_on,DATETIME,True,None,auto,0
6,text,VARCHAR(500),True,None,auto,0
7,tokenized,BLOB,True,None,auto,0
